In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

import re
import string
import nltk

from collections import Counter
from nltk import tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [2]:
df = pd.read_csv("../rawdata/hololivecomments.csv")
del df['subreddit.1']

In [3]:
df = df.dropna(axis=0, how="any", subset=None, inplace=False)

In [4]:
# Sort data berdasarkan waktu posting dan mengubah epoch value menjadi datetime UTC+7

df.sort_values(by = ["created_utc"], inplace = True, ascending = True)
df['created_utc'] = pd.DatetimeIndex(pd.to_datetime(df['created_utc'], unit='s')).tz_localize('UTC').tz_convert('Asia/Jakarta')

In [5]:
display(df.head(5))
display(df.tail(5))
display(df.info())

,subreddit,id,submission_id,created_utc,author,score,body,parent_id,permalink
406059,Hololive,ff4gful,erj4fp,2020-01-21 06:30:06+07:00,raystrife96,4.0,"I'm not up-to-date with Rushia, but I think th...",t3_erj4fp,/r/Hololive/comments/erj4fp/i_have_some_questi...
406061,Hololive,ff6nd9s,erj4fp,2020-01-22 01:21:24+07:00,raystrife96,2.0,"Yeah, if she changes to her default outfit, th...",t1_ff5k2n9,/r/Hololive/comments/erj4fp/i_have_some_questi...
406050,Hololive,ffhj7ub,etl9ue,2020-01-25 14:35:10+07:00,raystrife96,16.0,Okayu and Korone form a great pair too for cut...,t3_etl9ue,/r/Hololive/comments/etl9ue/what_pair_do_you_t...
406052,Hololive,ffhy6m7,etl9ue,2020-01-25 20:29:34+07:00,JustAnotherPep,3.0,"I haven't watched any of their stuff yet, I gu...",t1_ffhj7ub,/r/Hololive/comments/etl9ue/what_pair_do_you_t...
406051,Hololive,ffi49xe,etl9ue,2020-01-25 22:04:17+07:00,GuiMr27,3.0,Hmmmmmm... maybe FAMS? Idk,t3_etl9ue,/r/Hololive/comments/etl9ue/what_pair_do_you_t...


,subreddit,id,submission_id,created_utc,author,score,body,parent_id,permalink
2380082,Hololive,jadiweg,zd2bh2,2023-03-01 00:38:42+07:00,Chrisjared96,1.0,In the words of Mike Tyson which is very much ...,t1_iz3yflk,/r/Hololive/comments/zd2bh2/joint_statement/ja...
323369,Hololive,jafc2nw,i1078g,2023-03-01 07:52:04+07:00,DragonfruitAsleep976,1.0,Time scam every round: Warukunai yo ne\~,t3_i1078g,/r/Hololive/comments/i1078g/watame_is_in_granb...
478475,Hololive,jaffib4,klrhbh,2023-03-01 08:18:08+07:00,Electric-Guitar-9022,1.0,I think it because does the cartoon character ...,t3_klrhbh,/r/Hololive/comments/klrhbh/why_do_some_people...
406047,Hololive,jajp76x,ewewcp,2023-03-02 05:31:02+07:00,Erotically-Yours,1.0,"So I'm extremely late, and expect no response ...",t3_ewewcp,/r/Hololive/comments/ewewcp/i_love_haato_akais...
406048,Hololive,jand911,ewewcp,2023-03-03 00:50:44+07:00,Wackydude1234,1.0,I'm not sure if there is I've not been followi...,t1_jajp76x,/r/Hololive/comments/ewewcp/i_love_haato_akais...


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2285265 entries, 406059 to 406048
Data columns (total 9 columns):
 #   Column         Dtype                       
---  ------         -----                       
 0   subreddit      object                      
 1   id             object                      
 2   submission_id  object                      
 3   created_utc    datetime64[ns, Asia/Jakarta]
 4   author         object                      
 5   score          float64                     
 6   body           object                      
 7   parent_id      object                      
 8   permalink      object                      
dtypes: datetime64[ns, Asia/Jakarta](1), float64(1), object(7)
memory usage: 174.4+ MB


None

In [11]:
nltk.download('punkt')
nltk.download('stopwords')

STOP_WORDS = stopwords.words()

# removing the emojies
# https://www.kaggle.com/alankritamishra/covid-19-tweet-sentiment-analysis#Sentiment-analysis
EMOJI_PATTERN = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)


def cleaning(text):
    """
    Convert to lowercase.
    Rremove URL links, special characters and punctuation.
    Tokenize and remove stop words.
    """
    text = text.lower()
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('[’“”…]', '', text)

    text = EMOJI_PATTERN.sub(r'', text)

    # removing the stop-words
    text_tokens = word_tokenize(text)
    tokens_without_sw = [word for word in text_tokens if not word in STOP_WORDS]
    filtered_sentence = (" ").join(tokens_without_sw)
    text = filtered_sentence

    return text

[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
df2 = df.copy()

df2['body'] = df2['body'].apply(cleaning)
df2.to_csv("../rawdata/hololivecomments_cleaned.csv", index=False)